### Imports

In [1]:
#download data from https://www.kaggle.com/benhamner/nips-papers#papers.csv

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.nlp import *
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from torchtext import vocab, data, datasets
import pandas as pd
from pathlib import Path


In [2]:
sl=1000
vocab_size=200000
PATH=Path('../../data/language_modeling/nips_papers/')

TRN_PATH = PATH/'all/trn'
VAL_PATH = PATH/'all/val'

### Import and clean data

In [42]:
#nips_df = pd.read_csv(f'{PATH}/papers.csv')
print(nips_df.shape)
nips_df.head(30)

(7241, 7)


,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."
5,1002,1994,Using a neural net to instantiate a deformable...,NaN,1002-using-a-neural-net-to-instantiate-a-defor...,Abstract Missing,U sing a neural net to instantiate a\ndeformab...
6,1003,1994,Plasticity-Mediated Competitive Learning,NaN,1003-plasticity-mediated-competitive-learning.pdf,Abstract Missing,Plasticity-Mediated Competitive Learning\n\nTe...
7,1004,1994,ICEG Morphology Classification using an Analog...,NaN,1004-iceg-morphology-classification-using-an-a...,Abstract Missing,ICEG Morphology Classification using an\nAnalo...
8,1005,1994,Real-Time Control of a Tokamak Plasma Using Ne...,NaN,1005-real-time-control-of-a-tokamak-plasma-usi...,Abstract Missing,Real-Time Control of a Tokamak Plasma\nUsing N...
9,1006,1994,Pulsestream Synapses with Non-Volatile Analogu...,NaN,1006-pulsestream-synapses-with-non-volatile-an...,Abstract Missing,Real-Time Control of a Tokamak Plasma\nUsing N...


In [40]:
nips_df.title[1]

'A Mean Field Theory of Layer IV of Visual Cortex and Its Application to Artificial Neural Networks'

In [41]:
len(nips_df.title.unique())

7241

In [77]:
nips_df.abstract.unique()

array(['Abstract Missing',
       'Non-negative matrix factorization (NMF) has previously been shown to \r\nbe a useful decomposition for multivariate data. Two different multi- \r\nplicative algorithms for NMF are analyzed. They differ only slightly in \r\nthe multiplicative factor used in the update rules. One algorithm can be \r\nshown to minimize the conventional least squares error while the other \r\nminimizes the generalized Kullback-Leibler divergence. The monotonic \r\nconvergence of both algorithms can be proven using an auxiliary func- \r\ntion analogous to that used for proving convergence of the Expectation- \r\nMaximization algorithm. The algorithms can also be interpreted as diag- \r\nonally rescaled gradient descent, where the rescaling factor is optimally \r\nchosen to ensure convergence. ',
       'Spike-triggered averaging techniques are effective for linear characterization of neural responses. But neurons exhibit important nonlinear behaviors, such as gain control,

In [4]:
len(nips_df.abstract.unique())

3923

In [56]:
def get_abstract(paper_text):
    if(paper_text!=None):
        abstract_find=paper_text.find('ABSTRACT')
        if(abstract_find!=-1):
            return paper_text.split("ABSTRACT")[1].split("INTRODUCTION")[0].replace('\n','')
    else:
        return None

In [57]:
nips_df.abstract = nips_df.apply(lambda x: get_abstract(x.paper_text) if x.abstract == 'Abstract Missing' else x.abstract, axis=1)

In [58]:
nips_df.abstract[5100]

'Canonical Correlation Analysis (CCA) is a widely used statistical tool with both well established theory and favorable performance for a wide range of machine learning problems. However, computing CCA for huge datasets can be very slow since it involves implementing QR decomposition or singular value decomposition of huge matrices. In this paper we introduce L-CCA, an iterative algorithm which can compute CCA fast on huge sparse datasets. Theory on both the asymptotic convergence and finite time accuracy of L-CCA are established. The experiments also show that L-CCA outperform other fast CCA approximation schemes on two real datasets.'

In [59]:
len(nips_df.abstract.unique())

4197

In [64]:
def get_conclusion(paper_text):
    if(paper_text!=None):
        CONCLUSION_find=paper_text.find('CONCLUSION')
        if(CONCLUSION_find!=-1):
            return paper_text.split("CONCLUSION")[1].split("REFERENCES")[0].replace('\n','')
    else:
        return None

In [65]:
nips_df["conclusion"] = nips_df.paper_text.apply(get_conclusion)

In [66]:
nips_df.conclusion[0]

'A method of self-organizing associative databases was proposed with the application torobot eyesight systems. The machine decomposes a global structure unknown into a set oflocal structures known and learns universally any input-output response. This frameworkof problem implies a wide application area other than the examples shown in this paper.A defect of the algorithm 3 of self-organization is that the tree is balanced well onlyfor a subclass of structures of f. A subject imposed us is to widen the class. A probablesolution is to abolish the addressing rule depending directly on values of d and, instead, toestablish another rule depending on the distribution function of values of d. It is now underinvestigation.'

In [67]:
len(nips_df.conclusion.unique())

520

In [22]:
nips_df.to_csv(f'{PATH}/better_nips.csv')

In [68]:
nips_df.head(10)

,id,year,title,event_type,pdf_name,abstract,paper_text,conclusion
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,An efficient method of self-organizing associa...,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,A method of self-organizing associative databa...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,A single cell theory for the development of se...,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,None
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,In modeling studies or memory based on neural ...,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,SThese experiments identify a novel fono of an...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,None,Bayesian Query Construction for Neural\nNetwor...,None
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,None,"Neural Network Ensembles, Cross\nValidation, a...",The central idea in this paper was to show tha...
5,1002,1994,Using a neural net to instantiate a deformable...,NaN,1002-using-a-neural-net-to-instantiate-a-defor...,None,U sing a neural net to instantiate a\ndeformab...,None
6,1003,1994,Plasticity-Mediated Competitive Learning,NaN,1003-plasticity-mediated-competitive-learning.pdf,None,Plasticity-Mediated Competitive Learning\n\nTe...,By using the derivative of a logistic activati...
7,1004,1994,ICEG Morphology Classification using an Analog...,NaN,1004-iceg-morphology-classification-using-an-a...,None,ICEG Morphology Classification using an\nAnalo...,The successful classification of a difficult c...
8,1005,1994,Real-Time Control of a Tokamak Plasma Using Ne...,NaN,1005-real-time-control-of-a-tokamak-plasma-usi...,None,Real-Time Control of a Tokamak Plasma\nUsing N...,None
9,1006,1994,Pulsestream Synapses with Non-Volatile Analogu...,NaN,1006-pulsestream-synapses-with-non-volatile-an...,None,Real-Time Control of a Tokamak Plasma\nUsing N...,None


In [84]:
def get_txt(df):
    return ' <SUMM> ' + df.abstract + ' <TITLE> ' + df.title
nips_df['txt'] = get_txt(nips_df)
n=len(nips_df); n

7241

In [93]:
nips_txt=nips_df['txt'].dropna()
nips_txt.shape

(4198,)

In [94]:
nips_txt.head()

0      <SUMM> An efficient method of self-organizing...
1      <SUMM> A single cell theory for the developme...
2      <SUMM> In modeling studies or memory based on...
13     <SUMM> A large fraction of recent work in art...
24     <SUMM> This paper addresses the problem of de...
Name: txt, dtype: object

In [97]:
type(nips_txt)

pandas.core.series.Series

In [101]:
for (i,(_,r)) in enumerate(nips_txt.iteritems()):
    dset = 'trn' if random.random()>0.1 else 'val'
    f=open(f'{PATH}/all/{dset}/{i}.txt', 'w')
    #print(r)
    f.write(r)

In [78]:
os.makedirs(f'{PATH}/trn/yes', exist_ok=True)
os.makedirs(f'{PATH}/val/yes', exist_ok=True)
os.makedirs(f'{PATH}/trn/no', exist_ok=True)
os.makedirs(f'{PATH}/val/no', exist_ok=True)
os.makedirs(f'{PATH}/all/trn', exist_ok=True)
os.makedirs(f'{PATH}/all/val', exist_ok=True)
os.makedirs(f'{PATH}/models', exist_ok=True)

### load data into datatset

In [11]:
!find {PATH}/all/trn -name '*.txt' | xargs cat | wc -w

726222


In [12]:
!find {PATH}/all/val -name '*.txt' | xargs cat | wc -w

72841


In [3]:
TRN_PATH = 'trn'
VAL_PATH = 'val'


In [4]:
from spacy.symbols import ORTH

my_tok = spacy.load('en')

my_tok.tokenizer.add_special_case('<SUMM>', [{ORTH: '<SUMM>'}])
my_tok.tokenizer.add_special_case('<TITLE>', [{ORTH: '<TITLE>'}])
my_tok.tokenizer.add_special_case('<BR />', [{ORTH: '<BR />'}])
my_tok.tokenizer.add_special_case('<BR>', [{ORTH: '<BR>'}])

def my_spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(x)]

In [5]:
TEXT = data.Field(lower=True, tokenize=my_spacy_tok)
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)

In [9]:
from fastai.nlp import *
md = LanguageModelData.from_text_files(f'{PATH}/all/', TEXT, **FILES, bs=64, bptt=70, min_freq=10)

In [41]:
pickle.dump(TEXT, open(f'{PATH}/models/TEXT.pkl','wb'))

In [6]:
TEXT=pickle.load(open(f'{PATH}/models/TEXT.pkl','rb'))

In [10]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(192, 4832, 1, 865365)

In [8]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', 'the', ',', '.', 'of', 'a', '-', 'and', 'to', 'in', 'we']

In [11]:
' '.join(md.trn_ds[0].text[:150])


'  <summ> in real - world action recognition problems , low - level features can not adequately characterize the rich spatial - temporal structures in action videos . in this work , we encode actions based on attributes that describes actions as high - level concepts : \\textit{e.g . } , jump forward and motion in the air . we base our analysis on two types of action attributes . one type of action attributes is generated by humans . the second type is data - driven attributes , which is learned from data using dictionary learning methods . attribute - based representation may exhibit high variance due to noisy and redundant attributes . we propose a discriminative and compact attribute - based representation by selecting a subset of discriminative attributes from a large attribute set . three attribute selection criteria are proposed and formulated as a submodular optimization problem'

In [12]:
TEXT.numericalize([md.trn_ds[0].text[:12]])

Variable containing:
   15
   27
   10
  106
    7
  247
  377
  232
   79
    3
  169
    7
[torch.cuda.LongTensor of size 12x1 (GPU 0)]

In [14]:
next(iter(md.trn_dl))

(Variable containing:
    15   398   397  ...      2     5     6
    27    37   855  ...    449    29   617
    10  3301     3  ...    384    68    32
        ...          ⋱          ...       
   611  1992  2042  ...      8   732  1664
     5   531   144  ...    541    37   193
   377   941   313  ...   4744     0     3
 [torch.cuda.LongTensor of size 74x64 (GPU 0)], Variable containing:
    27
    37
   855
   ⋮  
    96
   478
     2
 [torch.cuda.LongTensor of size 4736 (GPU 0)])

In [15]:
x,y=next(iter(md.trn_dl))

In [16]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [17]:
from fastai.learner import *
learner = md.get_model(opt_fn, em_sz, nh, nl,
    dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [18]:
learner.fit(3e-3, 1, wds=1e-6)


epoch      trn_loss   val_loss                              
    0      5.349139   5.0534    



[array([5.0534])]

In [19]:
learner.fit(3e-3, 3, wds=1e-6, cycle_len=1, cycle_mult=2)


epoch      trn_loss   val_loss                              
    0      4.855386   4.748243  
    1      4.634948   4.496106                              
    2      4.453592   4.413718                              
    3      4.402725   4.304138                              
    4      4.247719   4.204061                              
    5      4.115151   4.154432                              
    6      4.060921   4.142037                              



[array([4.14204])]

In [20]:
learner.save_encoder('adam2_enc')


In [21]:
learner.fit(3e-3, 10, wds=1e-6, cycle_len=5, cycle_save_name='adam3_10')


epoch      trn_loss   val_loss                              
    0      4.141044   4.149134  
    1      4.03629    4.083334                              
    2      3.930122   4.055129                              
    3      3.851135   4.036102                              
    4      3.800695   4.036169                              
    5      3.929968   4.044878                              
    6      3.847969   4.021999                              
    7      3.75177    4.00745                               
    8      3.684062   3.997845                              
    9      3.63479    4.004062                              
    10     3.785001   4.01344                               
    11     3.72564    3.99274                               
    12     3.6807     3.978439                              
    13     3.562742   3.978908                              
    14     3.511159   3.998276                              
    15     3.678425   3.998196                      

[array([4.08481])]

In [22]:
learner.save_encoder('adam3_10_enc')

In [ ]:
learner.fit(3e-3, 8, wds=1e-6, cycle_len=10, cycle_save_name='adam3_5')
learner.save_encoder('adam3_20_enc')
learner.save('adam3_20')


### Test

In [24]:
def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)])


In [25]:
m=learner.model

In [28]:
def sample_model(m, s, l=50):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    print('...', end='')

    for i in range(l):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        print(word, end=' ')
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0))

    m[0].bs=batch_sz

In [29]:
sample_model(m,"<SUMM> An efficient method of self-organizing")

...neural network architecture is proposed . the network is trained to be a " real world " of the input data , and the network is trained to be a " real world " of the input data . the network is trained to be a " real world " 

In [30]:
sample_model(m,"<TITLE> Bayesian Query Construction for Neural Network")

...planning <eos> 

In [31]:
sample_model(m,"<SUMM> Non-negative matrix factorization (NMF) has previously been shown")

...to be a powerful and powerful tool for modeling the structure of data in a latent space .   in this paper , we propose a novel approach to learning the parameters of a mixture model , where the model is able to learn a sparse gaussian process ( gp 

In [32]:
sample_model(m,"<SUMM> Spike-triggered averaging techniques are effective for linear characterization of neural responses. But neurons exhibit important nonlinear behaviors, such as gain control, that are not captured by such analyses")

.... we present a novel approach to learning the parameters of the model using a gaussian process prior over the posterior distribution . we show that the model can be used to learn the parameters of a markov chain , and we show that the resulting model can be used 

In [34]:
sample_model(m,"<TITLE> Neural Network Ensembles, cross ")

...- validation analysis of neural networks <eos> 

In [35]:
sample_model(m,"<TITLE> Convolutional Neural Network , ")

...with application to brain activity analysis <eos> 

In [36]:
sample_model(m,"<SUMM> storing-covariance-by-the-associative")

...- known models in a variety of applications , such as text classification and text classification , is often limited by the lack of a number of methods to train the models . in this paper , we propose a novel approach to learning the parameters of a neural network 

In [45]:
sample_model(m,"<TITLE> Correlated Neuronal Response:")

...a unified theory for high - dimensional neural networks <eos> 

In [46]:
sample_model(m,"<TITLE> Learning to Play the Game of ")

...: a unified approach to stochastic optimization <eos> 